In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import IPython.display

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pymedphys
import pymedphys._wlutz.iview
import pymedphys._wlutz.imginterp
import pymedphys._wlutz.findfield
import pymedphys._wlutz.findbb
import pymedphys._wlutz.reporting

import pymedphys._vendor.pylinac.winstonlutz

In [ ]:
image_path = pymedphys.data_path('wlutz_image.png')

In [ ]:
penumbra = 2
edge_lengths = [20, 20]
initial_rotation = 0
bb_diameter = 8

pd.set_option("display.max_rows", 101)

In [ ]:
x, y, img = pymedphys._wlutz.iview.iview_image_transform(image_path)

field = pymedphys._wlutz.imginterp.create_interpolated_field(x, y, img)
initial_centre = pymedphys._wlutz.findfield._initial_centre(x, y, img)


field_centre, field_rotation = pymedphys._wlutz.findfield.field_centre_and_rotation_refining(
    field, edge_lengths, penumbra, initial_centre, initial_rotation=initial_rotation
)

field_centre = np.round(field_centre, decimals=2).tolist()
field_rotation = np.round(field_rotation, decimals=1)

bb_centre = pymedphys._wlutz.findbb.optimise_bb_centre(
    field, bb_diameter, edge_lengths, penumbra, field_centre, field_rotation
)

bb_centre = np.round(bb_centre, decimals=2).tolist()

In [ ]:
bb_centre

In [ ]:
field_centre

In [ ]:
centralised_straight_field = pymedphys._wlutz.findbb.create_centralised_field(field, field_centre, field_rotation)
half_x_range = edge_lengths[0]/2 + penumbra * 3
half_y_range = edge_lengths[1]/2 + penumbra * 3

pixel_size = 0.1

x_range = np.arange(-half_x_range, half_x_range + pixel_size, pixel_size)
y_range = np.arange(-half_y_range, half_y_range + pixel_size, pixel_size)

xx_range, yy_range = np.meshgrid(x_range, y_range)
centralised_image = centralised_straight_field(xx_range, yy_range)

In [ ]:
half_x_range

In [ ]:
np.shape(centralised_image)

In [ ]:
pylinac_wlutz = pymedphys._vendor.pylinac.winstonlutz.WLImage(centralised_image)

In [ ]:
centralised_pylinac_field_centre = [
    pylinac_wlutz.field_cax.x * pixel_size - half_x_range,
    pylinac_wlutz.field_cax.y * pixel_size - half_y_range
]

centralised_pylinac_bb_centre = [
    pylinac_wlutz.bb.x * pixel_size - half_x_range,
    pylinac_wlutz.bb.y * pixel_size - half_y_range
]

In [ ]:
centralised_pylinac_field_centre

In [ ]:
pylinac_field_centre = pymedphys._wlutz.findbb.transform_bb_centre(centralised_pylinac_field_centre, field_centre, field_rotation)
pylinac_bb_centre = pymedphys._wlutz.findbb.transform_bb_centre(centralised_pylinac_bb_centre, field_centre, field_rotation)

pylinac_field_centre = np.round(pylinac_field_centre, decimals=2).tolist()
pylinac_bb_centre = np.round(pylinac_bb_centre, decimals=2).tolist()

In [ ]:
pylinac_bb_centre

In [ ]:
pylinac_field_centre

In [ ]:
field_centre

In [ ]:
bb_centre

In [ ]:
fig = pymedphys._wlutz.reporting.image_analysis_figure(
    x,
    y,
    img,
    bb_centre,
    field_centre,
    field_rotation,
    bb_diameter,
    edge_lengths,
    penumbra,
)

In [ ]:
bb_bounds = pymedphys._wlutz.findbb.define_bb_bounds(bb_diameter, edge_lengths, penumbra)
bb_bounds

In [ ]:
bb_centre = (0, -7.5)
pymedphys._wlutz.findbb.check_if_at_bounds(bb_centre, bb_bounds)

In [ ]:
penumbra = 2
edge_lengths = [20, 24]
bb_diameter = 8

bb_bounds = pymedphys._wlutz.findbb.define_bb_bounds(
    bb_diameter, edge_lengths, penumbra
)
assert pymedphys._wlutz.findbb.check_if_at_bounds((0, -7.5), bb_bounds)
assert not pymedphys._wlutz.findbb.check_if_at_bounds(
    (0, -5.5), bb_bounds
)
assert pymedphys._wlutz.findbb.check_if_at_bounds((5.5, 0), bb_bounds)

In [ ]:
np.any(np.array(bb_centre[0]) == np.array(bb_bounds[0]))
np.any(np.array(bb_centre[1]) == np.array(bb_bounds[1]))

In [ ]:
np.any(np.array(bb_centre)[:,None,None] == np.array(bb_bounds)[None,:,:])